### 2015-02 2do Recuperatorio

Un telescopio registra automaticamente la luminosidad de distintas
estrellas generando un RDD con registros de tipo (star_id,
magnitude,spectral_type, timestamp). Queremos obtener un listado de
estrellas que tienen el mismo tipo espectral e igual promedio de
magnitud una vez redondeado el mismo a un decimal. El resultado
debe ser una lista en donde cada elemento de la lista es una lista de ids
de estrellas similares.

In [1]:
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [2]:
data = sc.parallelize(stars);

In [3]:
data.collect()

[(1, 5, 1, 1),
 (2, 10, 1, 1),
 (3, 6, 1, 1),
 (4, 5.5, 2, 1),
 (1, 6, 1, 2),
 (2, 9, 1, 2),
 (3, 5, 1, 2),
 (1, 5.5, 1, 3),
 (2, 11, 1, 3),
 (3, 5.5, 1, 3)]

In [4]:
data.map(lambda x: (x[0], (x[2], x[1], 1)))\
    .reduceByKey(lambda x, y: (x[0], x[1]+y[1], x[2]+y[2]))\
    .map(lambda x: ((x[1][0], x[1][1]/x[1][2]), x[0]))\
    .groupByKey()\
    .map(lambda x: (x[0], list(x[1])))\
    .collect()
    
# El último map es simplemente para convertir la sequencia de spark en una lista de python para poder verla

[((1, 5.5), [1, 3]), ((1, 10.0), [2]), ((2, 5.5), [4])]

### 2015-01 2do Recuperatorio

Tenemos una colección de documentos (textos) almacenados en un cluster. Se quiere establecer un ranking de los patrones mas frecuentes para la aparición de letras en las palabras. Por ejemplo “sister”, “ejects” , “ninety” y “amazon” responden al patrón “abacde”. Programar en map-reduce un programa que genere como resultado un
listado de tipo (patron, frecuencia) indicando cuántas veces aparece cada patrón en la colección de documentos.

In [23]:
textFile = sc.textFile("data/shakespeare.txt")
words = textFile.flatMap(lambda line: line.split()).map(lambda word: word.lower())
words.take(5)

['1609', 'the', 'sonnets', 'by', 'william']

In [24]:
def pattern(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    pat = ''
    found = ''
    for letter in word:
        if letter in found:
            pat += pat[found.index(letter)]
        else:
            found += letter
            if len(found) > len(letters):
                pat += '?'
            else:
                pat += letters[len(found)-1]
    return pat
   
words.map(lambda x: (pattern(x), 1))\
     .reduceByKey(lambda x, y: x+y)\
     .takeOrdered(10, lambda x: -x[1])

[('abc', 165274),
 ('abcd', 146402),
 ('ab', 138768),
 ('abcde', 83840),
 ('abcdef', 47396),
 ('a', 35512),
 ('abcdefg', 26066),
 ('abca', 13868),
 ('abcdefgh', 13800),
 ('abcc', 13287)]

### 2016-02 Parcial

En este ejercicio queremos programar un sistema que recomiende textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto es un string de longitud variable.

Además contamos con un RDD que indica qué términos le gustan o no a cada usuario de la forma (userId, término, score) por ejemplo (23, “calesita”, -2).

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para los términos que aparecen en el documento.

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en estos casos simplemente los consideramos neutros (score=0)

In [25]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [6]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [9]:
# generamos registros del tipo (word, docId) donde word sera nuestra key
words = documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])
words.collect()

[('pablo', 1),
 ('honey', 1),
 ('the', 2),
 ('bends', 2),
 ('ok', 3),
 ('computer', 3),
 ('kid', 4),
 ('a', 4),
 ('amnesiac', 5),
 ('hail', 6),
 ('to', 6),
 ('the', 6),
 ('thief', 6),
 ('in', 7),
 ('rainbows', 7),
 ('the', 8),
 ('king', 8),
 ('of', 8),
 ('limbs', 8),
 ('a', 9),
 ('moon', 9),
 ('shaped', 9),
 ('pool', 9)]

In [8]:
# llevamos scores a una representacion donde word sea la clave
scoresRDDForJoin = scoresRDD.map(lambda a: (a[1],(a[0],a[2])))
scoresRDDForJoin.collect()

[('pablo', ('thom', 1)),
 ('honey', ('thom', 1)),
 ('pablo', ('martin', -1)),
 ('honey', ('martin', -1)),
 ('ok', ('martin', 30)),
 ('computer', ('martin', 30))]

In [13]:
words_join_scores = words.join(scoresRDDForJoin)
words_join_scores.collect()

[('honey', (1, ('thom', 1))),
 ('honey', (1, ('martin', -1))),
 ('ok', (3, ('martin', 30))),
 ('pablo', (1, ('thom', 1))),
 ('pablo', (1, ('martin', -1))),
 ('computer', (3, ('martin', 30)))]

In [14]:
# para poder realizar una acumulacion debemos llevar la informacion a la representacion ((docId, userId), score) 
words_join_scores_mapped = words_join_scores.map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))
words_join_scores_mapped.collect()

[((1, 'thom'), 1),
 ((1, 'martin'), -1),
 ((3, 'martin'), 30),
 ((1, 'thom'), 1),
 ((1, 'martin'), -1),
 ((3, 'martin'), 30)]

In [16]:
# acumulamos y llevamos a la representación pedida
words_join_scores_mapped.reduceByKey(lambda a,b: a + b)\
                        .map(lambda a: (a[0][0], a[0][1], a[1]))\
                        .collect()

[(3, 'martin', 60), (1, 'martin', -2), (1, 'thom', 2)]

### 2017-01 Parcial

Se tiene información estadística de la temporada regular de todos los jugadores de la NBA en un RDD de tuplas con el siguiente formato: (id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos, promedio_bloqueos, promedio_rebotes, promedio_faltas).

Un analista de la cadena ESPN está trabajando con un RDD que corresponde a la primera ronda de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp, cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos, cantidad_asistencias, cantidad_faltas).

En base a estos RDDs se quiere programar en PySpark un programa que genere un RDD con los nombres (sin duplicados) de los jugadores que lograron en algún partido de playoffs una cantidad de asistencias mayor a su promedio histórico.

##### Criterio

Existen diversas maneras de realizarlo, pero es posible resolverlo realizando un reduceByKey con clave (id_jugador,id_partido) acumulando la cantidad de asistencias por partido quedando ((id_jugador, id_partido), sum_asistencias).

Luego efectuamos un cambio de clave a id_jugador con un map para poder realizar un join de la informacion de la temporada usando id_jugador.

Una vez hecho esto, filtramos aquella informacion que cumple de mayor cantidad asistencias en el partido que el promedio de la temporada.

Dado que pueden quedarnos duplicados (un jugador podria superar su promedio de temporada en varios partidos), es importante al mapearla informacion al formato final realizar un distinct.

In [10]:
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

players_all_time_stats_rdd = sc.parallelize(players_all_time_stats)
scores_rdd = sc.parallelize(scores)

In [11]:
# cambio de clave a id_jugador, partido
scores_by_match = scores_rdd.map(lambda a: ((a[0],a[1]),a[3]))
scores_by_match = scores_by_match.reduceByKey(lambda a,b: a+b)
scores_by_match.collect()

[((1, 1), 200), ((1, 2), 300), ((2, 1), 300), ((3, 2), 100)]

In [12]:
# cambio de clave a id_jugador
scores_by_player = scores_by_match.map(lambda a: (a[0][0], a[1]))
scores_by_player.collect()

[(1, 200), (1, 300), (2, 300), (3, 100)]

In [13]:
# juntamos los datos para poder evaluar lo pedido
# preparamos la key del lado de informacion historica de jugadores.

all_time_with_key = players_all_time_stats_rdd.map(lambda a: (a[0],a))
all_time_with_key.collect()

[(1, (1, 'Manu Ginobili', 800)),
 (2, (2, 'Kobe Bryant', 100)),
 (3, (3, 'Marc Gasol', 25)),
 (4, (4, 'James Harden', 1000))]

In [14]:
scores_by_player = scores_by_player.join(all_time_with_key)
scores_by_player.collect()

[(1, (200, (1, 'Manu Ginobili', 800))),
 (1, (300, (1, 'Manu Ginobili', 800))),
 (2, (300, (2, 'Kobe Bryant', 100))),
 (3, (100, (3, 'Marc Gasol', 25)))]

In [15]:
overperformers = scores_by_player.filter(lambda a: a[1][0] > a[1][1][2])
overperformers.collect()

[(2, (300, (2, 'Kobe Bryant', 100))), (3, (100, (3, 'Marc Gasol', 25)))]

In [16]:
# veamos los duplicados
overperformers = overperformers.map(lambda a: a[1][1][1]).distinct()
overperformers.collect()

['Marc Gasol', 'Kobe Bryant']